####### Se cuenta con estaciones de metro de todo México a través del tiempo y nos solicitan resolver dos problemáticas:

- 1) Pronosticar la afluencia en cualquier estación
- 2) Identificar si la estación estará en alta demanda ( si tiene un incremento de afluencia de 4 veces o más consecutivas )

# Librerías

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ks_2samp
from varclushi import VarClusHi
from sklearn.feature_selection import VarianceThreshold 
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from scikitplot.metrics import plot_roc_curve

# Extracción de datos

In [2]:
ruta = '/home/luis/Documentos/entornos/Ciencia de datos_git/Clasificaciones_estimaciones'

In [3]:
lst_archi = os.listdir(ruta)

In [4]:
lst_archi = [os.path.join( ruta , f ) for f in lst_archi  if f[-3:] == 'csv'  ]

In [5]:
df = pd.read_csv(lst_archi[0], sep='|')

In [6]:
df.sample(10)

,afluencia,id_estacion,t
61129,133075,Est00160,57
239705,231348,Est00074,291
149431,48811,Est00002,431
34891,29973,Est00244,163
197891,78773,Est00360,454
95062,6104,Est00009,457
122474,88076,Est00349,669
42429,113585,Est00131,352
184199,405,Est00218,320
53611,148301,Est00213,553


# Limpieza / Pretratamiento

In [7]:
df.dtypes

afluencia       int64
id_estacion    object
t               int64
dtype: object

In [8]:
df.id_estacion.value_counts()

Est00127    700
Est00116    700
Est00037    700
Est00197    700
Est00195    700
           ... 
Est00256    699
Est00356    699
Est00094    699
Est00034    699
Est00320    699
Name: id_estacion, Length: 390, dtype: int64

In [9]:
df.id_estacion.value_counts(True)*100

Est00127    0.256526
Est00116    0.256526
Est00037    0.256526
Est00197    0.256526
Est00195    0.256526
              ...   
Est00256    0.256159
Est00356    0.256159
Est00094    0.256159
Est00034    0.256159
Est00320    0.256159
Name: id_estacion, Length: 390, dtype: float64

In [10]:
df

,afluencia,id_estacion,t
0,137957,Est00127,1
1,343181,Est00127,2
2,454925,Est00127,5
3,393558,Est00127,7
4,336768,Est00127,13
...,...,...,...
272872,405,Est00320,680
272873,405,Est00320,693
272874,405,Est00320,694
272875,405,Est00320,695


# Variables

In [11]:
um = ['id_estacion']
t_min , t_max = df.t.min() , df.t.max()
cols = range(t_min , t_max+1)

In [12]:
t_min, t_max

(1, 700)

In [13]:
df.shape

(272877, 3)

In [14]:
df_piv = df.pivot_table( index=um , columns='t', values = 'afluencia' ).reset_index()

In [15]:
df_piv

t,id_estacion,1,2,3,4,5,6,7,8,9,...,691,692,693,694,695,696,697,698,699,700
0,Est00000,39059.0,74700.0,44356.0,237597.0,235918.0,181140.0,257563.0,257761.0,142208.0,...,370097.0,305425.0,311695.0,325139.0,240916.0,110586.0,394055.0,343256.0,374973.0,359923.0
1,Est00001,48321.0,368193.0,293510.0,295068.0,200780.0,236938.0,152211.0,283677.0,114380.0,...,97849.0,51293.0,93265.0,116890.0,116176.0,109783.0,123092.0,79045.0,66206.0,94625.0
2,Est00002,17515.0,49586.0,49395.0,42017.0,27076.0,38617.0,33000.0,43166.0,20616.0,...,18127.0,12728.0,18583.0,21357.0,21351.0,20371.0,22105.0,19344.0,12476.0,17916.0
3,Est00003,40677.0,240827.0,223303.0,190028.0,148294.0,148437.0,220882.0,253680.0,76196.0,...,68953.0,55081.0,102738.0,92429.0,102303.0,101997.0,94795.0,90334.0,26519.0,NaN
4,Est00004,82758.0,163112.0,144023.0,240623.0,243391.0,205486.0,220086.0,255863.0,205819.0,...,288845.0,256489.0,249879.0,276292.0,292926.0,221677.0,308573.0,289825.0,310192.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,Est00385,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,...,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0
386,Est00386,19378.0,29735.0,23336.0,41908.0,41548.0,43017.0,40262.0,46648.0,33353.0,...,68511.0,64390.0,58222.0,63731.0,71912.0,42255.0,69844.0,64880.0,70300.0,64261.0
387,Est00387,146778.0,368635.0,302365.0,411511.0,382508.0,313966.0,317081.0,410328.0,390995.0,...,269899.0,275897.0,272368.0,297006.0,336441.0,277761.0,295551.0,289471.0,300052.0,NaN
388,Est00388,70477.0,119392.0,108627.0,162861.0,244248.0,177978.0,192803.0,219787.0,156910.0,...,280277.0,239542.0,229206.0,246336.0,211212.0,140453.0,267132.0,258828.0,275850.0,266479.0


# Funciones a utilizar

In [16]:
def metricas(model,Xv,yv):
    print(" Métricas para modelo de clasificación: \n")

    print(" Valor ROC : %.3f"   %roc_auc_score( y_score=model.predict_proba(Xv)[:,1] , y_true=yv  )   )

    print(" Valor ACC : %.3f\n" %accuracy_score( y_pred=model.predict(Xv) , y_true=yv) )

    print(" Matriz de confusión: ", "\n", confusion_matrix(y_pred=model.predict(Xv) , y_true=yv ) )

# Funciones de agregación

In [17]:
def sum_inc(l):
    return sum( [int(y>x) for x,y in zip( l , l[1:] ) ] )

def sum_dec(l):
    return sum( [int(y<x) for x,y in zip( l , l[1:] ) ] )    

def media_inc(l):
    return np.mean( [int(y>x) for x,y in zip( l , l[1:] ) ] )

def media_dec(l):
    return np.mean( [int(y<x) for x,y in zip( l , l[1:] ) ] )  

def delta_min(l):
    try:
        return min( [ float( y-x ) for x,y in zip( l , l[1:] ) ] )
    except:
        return np.nan
    
def delta_max(l):
    try:
        return max( [ float( y-x ) for x,y in zip( l , l[1:] ) ] )
    except:
        return np.nan

def delta_mean(l):
    try:
        return np.mean( [ float( y-x ) for x,y in zip( l , l[1:] ) ] )
    except:
        return np.nan
    
def delta_desv(l):  # revisar
    try:
        return np.std( [ float( y-x ) for x,y in zip( l , l[1:] ) ] )
    except:
        return np.nan
    
def pct_delta_min(l):
    try:
        return np.min( [ float( y-x )/x for x,y in zip( l , l[1:] ) ] )
    except:
        return np.nan
    
def pct_delta_max(l):
    try:
        return np.max( [ float( y-x )/x for x,y in zip( l , l[1:] ) ] )
    except:
        return np.nan
    
def pct_delta_mean(l):
    try:
        return np.mean( [ float( y-x )/x for x,y in zip( l , l[1:] ) ] )
    except:
        return np.nan
    
def pct_delta_desv(l): # revisar
    try:
        return np.std( [ float( y-x )/x for x,y in zip( l , l[1:] ) ] )
    except:
        return np.nan
    
def max_racha_inc(l):
    return max( [ len(i) for i in "".join([ str( int(y>x)  ) for x,y in zip(l,l[1:])  ]).split('0')  ]   )

def max_racha_dec(l):
    return max( [ len(i) for i in "".join([ str( int(y<x)  ) for x,y in zip(l,l[1:])  ]).split('0')  ]   )

def media_racha_inc(l):
    return np.mean( [ len(i) for i in "".join([ str( int(y>x)  ) for x,y in zip(l,l[1:])  ]).split('0')  ]   )

def media_racha_dec(l):
    return np.mean( [ len(i) for i in "".join([ str( int(y<x)  ) for x,y in zip(l,l[1:])  ]).split('0')  ]   )

####### Ejemplo igual al excel

In [18]:
l = [963,411,474,560,600,500,525,188,685]

In [19]:
pct_delta_mean(l)

0.21474808901015885

In [20]:
pct_delta_max(l)

2.643617021276596

####### Termina ejemplo de excel

In [21]:
lst_func = ['sum','std','min','mean','max',sum_inc,sum_dec,
media_inc, media_dec,
delta_min, delta_max,
delta_mean, delta_desv,
pct_delta_min, pct_delta_max,
pct_delta_mean, pct_delta_desv,
max_racha_inc, max_racha_dec,
media_racha_inc, media_racha_dec  ]

# Ventanas de tiempo

In [22]:
vdes = 1
vobs = 10

anclai = t_min + vobs -1
anclaf = t_max - vdes

In [23]:
vdes , vobs , anclai , anclaf

(1, 10, 10, 699)

# Matriz de predictoras

In [ ]:
lst_aux = []

ancla = anclai + vobs

for ancla in range( anclai , anclaf + 1 ):
    
    print('intervalo: ', ancla - anclai , ancla , " Para pronosticar: ", ancla+vdes)
    
    aux = df[ ( df['t'] > ancla-anclai ) & 
               (df['t'] <= ancla)  ].reset_index(drop=True).copy()
    
    aux = aux.pivot_table( index = um,
                           values = 'afluencia',
                           aggfunc = lst_func )
     
    aux.columns = [f'v_{i}_{j}' for i,j in aux.columns]
    
    aux.insert(0,'ancla',ancla)
    
    aux.reset_index(inplace=True)
    
    lst_aux.append(aux)
    
X = pd.concat( lst_aux , ignore_index=True ).copy()

intervalo:  0 10  Para pronosticar:  11
intervalo:  1 11  Para pronosticar:  12
intervalo:  2 12  Para pronosticar:  13
intervalo:  3 13  Para pronosticar:  14
intervalo:  4 14  Para pronosticar:  15
intervalo:  5 15  Para pronosticar:  16
intervalo:  6 16  Para pronosticar:  17
intervalo:  7 17  Para pronosticar:  18
intervalo:  8 18  Para pronosticar:  19
intervalo:  9 19  Para pronosticar:  20
intervalo:  10 20  Para pronosticar:  21
intervalo:  11 21  Para pronosticar:  22
intervalo:  12 22  Para pronosticar:  23
intervalo:  13 23  Para pronosticar:  24
intervalo:  14 24  Para pronosticar:  25
intervalo:  15 25  Para pronosticar:  26
intervalo:  16 26  Para pronosticar:  27
intervalo:  17 27  Para pronosticar:  28
intervalo:  18 28  Para pronosticar:  29
intervalo:  19 29  Para pronosticar:  30
intervalo:  20 30  Para pronosticar:  31
intervalo:  21 31  Para pronosticar:  32
intervalo:  22 32  Para pronosticar:  33
intervalo:  23 33  Para pronosticar:  34
intervalo:  24 34  Para pr

intervalo:  193 203  Para pronosticar:  204
intervalo:  194 204  Para pronosticar:  205
intervalo:  195 205  Para pronosticar:  206
intervalo:  196 206  Para pronosticar:  207
intervalo:  197 207  Para pronosticar:  208
intervalo:  198 208  Para pronosticar:  209
intervalo:  199 209  Para pronosticar:  210
intervalo:  200 210  Para pronosticar:  211
intervalo:  201 211  Para pronosticar:  212
intervalo:  202 212  Para pronosticar:  213
intervalo:  203 213  Para pronosticar:  214
intervalo:  204 214  Para pronosticar:  215
intervalo:  205 215  Para pronosticar:  216
intervalo:  206 216  Para pronosticar:  217
intervalo:  207 217  Para pronosticar:  218
intervalo:  208 218  Para pronosticar:  219
intervalo:  209 219  Para pronosticar:  220
intervalo:  210 220  Para pronosticar:  221
intervalo:  211 221  Para pronosticar:  222
intervalo:  212 222  Para pronosticar:  223
intervalo:  213 223  Para pronosticar:  224
intervalo:  214 224  Para pronosticar:  225
intervalo:  215 225  Para pronos

intervalo:  380 390  Para pronosticar:  391
intervalo:  381 391  Para pronosticar:  392
intervalo:  382 392  Para pronosticar:  393
intervalo:  383 393  Para pronosticar:  394
intervalo:  384 394  Para pronosticar:  395
intervalo:  385 395  Para pronosticar:  396
intervalo:  386 396  Para pronosticar:  397
intervalo:  387 397  Para pronosticar:  398
intervalo:  388 398  Para pronosticar:  399
intervalo:  389 399  Para pronosticar:  400
intervalo:  390 400  Para pronosticar:  401
intervalo:  391 401  Para pronosticar:  402
intervalo:  392 402  Para pronosticar:  403
intervalo:  393 403  Para pronosticar:  404
intervalo:  394 404  Para pronosticar:  405
intervalo:  395 405  Para pronosticar:  406
intervalo:  396 406  Para pronosticar:  407
intervalo:  397 407  Para pronosticar:  408
intervalo:  398 408  Para pronosticar:  409
intervalo:  399 409  Para pronosticar:  410
intervalo:  400 410  Para pronosticar:  411
intervalo:  401 411  Para pronosticar:  412
intervalo:  402 412  Para pronos

In [ ]:
X.head()

In [ ]:
X.to_pickle("datosEstaciones/X_aux.pkl",protocol=4)

# Vector solución

In [ ]:
lst_aux = []

for ancla in range( anclai , anclaf + 1 ):
    
    print('intervalo: ', ancla - anclai , ancla , " Para pronosticar: ", ancla+vdes)
    
    aux = df[ ( df['t'] > ancla-anclai ) & (df['t'] <= ancla + vdes)  ].reset_index(drop=True).copy()
    
    aux = aux.pivot_table( index = um,
                           columns = 't',
                           values = 'afluencia')
    
    aux['y'] = aux[ ancla + vdes ]
    
    aux = aux[['y']]
    
    aux.insert(0,'ancla',ancla)
    
    aux.reset_index(inplace=True)
    
    lst_aux.append(aux)
    
y = pd.concat( lst_aux , ignore_index=True )

In [ ]:
y

# TAD preliminar 

In [ ]:
tad = X.merge( y , on = ['id_estacion','ancla'] , how='inner')

In [ ]:
tad.head()

In [ ]:
tad.shape

# Target 2  - Para el álta demanda

In [ ]:
tad['v_max_racha_inc_afluencia' ].value_counts()

In [ ]:
tad['v_max_racha_inc_afluencia' ].value_counts(True)*100

In [ ]:
tad.apply( lambda x: (x['v_max_racha_inc_afluencia'] >= 3) , axis=True ).astype(int).value_counts(True)*100

In [ ]:
tad['y2'] = tad.apply( lambda x: (x['v_max_racha_inc_afluencia'] >= 3) , axis=True ).astype(int)

In [ ]:
tad.head()

# Persistir TAD

In [ ]:
tad.to_pickle( 'datosEstaciones/tad_preliminar.pkl',protocol=4)

# Análisis exploratorio

## Variables Continuas

In [ ]:
varc = tad.filter(like='v_').columns.tolist()

In [ ]:
len(varc), varc

In [ ]:
X = tad[varc].copy()

In [ ]:
X[varc].describe()

In [ ]:
X[varc[:9]].hist(figsize=(10,10))

### Valores ausentes

In [ ]:
miss = 1 - X[varc].describe().T[['count']] / len(tad)

In [ ]:
miss

In [ ]:
X.shape , X.dropna().shape , X.dropna().shape[0] / X.shape[0]

### Impuntación

In [ ]:
# Esto puede ser tan complejo o sofisticado como negocio o la variable lo requiera
im = SimpleImputer(strategy='median')

In [ ]:
im.fit(X)

In [ ]:
X[varc] = im.transform(X[varc])

### Distribución alterada

In [ ]:
ks = pd.DataFrame( map( lambda v:  ( v , ks_2samp( tad[v].dropna()  , X[v] ).statistic  ), varc ) , columns=['var','ks'])

In [ ]:
## Valores mayores a .1 son dist. alteradas

In [ ]:
ks

### Varianza 

In [ ]:
vt = VarianceThreshold(threshold=1)

In [ ]:
vt.fit( X[varc] )

In [ ]:
sin_varianza = [ v for v,u in zip(varc , vt.get_support() ) if not(u) ]

In [ ]:
sin_varianza

In [ ]:
X[sin_varianza].hist(figsize=(10,10))

In [ ]:
X.drop( sin_varianza , axis = 1  , inplace=True)

In [ ]:
varc = [v for v in varc if v not in sin_varianza]

In [ ]:
len(varc), varc

### Extremos

In [ ]:
ext = X[varc].describe(percentiles=[0.01,0.99]).T[['1%','99%']].reset_index()

In [ ]:
for v, li, ls in ext.values:
    X[f'ol_{v}'] = ( ( X[v] < li ) | (X[v] > ls) ).astype(int)
    
X['ext'] = X.filter(like='ol_').max(axis=1)
X.drop(X.filter(like='ol_').columns , axis=1 , inplace=True)
X['ext'].value_counts(True)

In [ ]:
X.shape , tad.shape

In [ ]:
X[ um + ['ancla'] ] = tad[ um + ['ancla']]

In [ ]:
X = X.loc[ X['ext'] == 0 ].reset_index(drop=True).drop(['ext'],axis=1)

In [ ]:
X.shape , tad.shape

### Multivariado

In [ ]:
X[varc].corr()

In [ ]:
sns.pairplot(X[varc].sample(500) )

## Multicolinealidad

In [ ]:
vc = VarClusHi( df = X[varc] , feat_list=varc )

In [ ]:
vc.varclus()

In [ ]:
rs = vc.rsquare

In [ ]:
rs = rs.sort_values(by=['Cluster','RS_Ratio'],ascending=[1,1]).reset_index(drop=True)

In [ ]:
rs['id'] = rs.groupby('Cluster').cumcount()+1

In [ ]:
rs

In [ ]:
varc = rs.loc[ rs.id == 1 ]['Variable'].tolist()

In [ ]:
varc

In [ ]:
X[varc].hist()

# TAD Final - Modelo 1

In [ ]:
X[ um +['ancla'] + varc ].head()

In [ ]:
y.head()

In [ ]:
tad_1 = X[ um + ['ancla'] + varc ].merge( y , on=[ 'id_estacion', 'ancla'] , how='inner').reset_index(drop=True)

In [ ]:
tad_1

In [ ]:
tad_2 = X[ um + ['ancla'] + varc ].merge( tad[um+['ancla','y2']] , 
                                          on=[ 'id_estacion', 'ancla'] , 
                                          how='inner').reset_index(drop=True)

In [ ]:
tad_2

# Modelo (1) : Regresión

In [ ]:
tad_1.shape , tad_1.loc[ ~tad_1['y'].isna() ].shape

In [ ]:
tad_1 = tad_1.loc[ ~tad_1['y'].isna() ].reset_index(drop=True)

In [ ]:
tad_1

In [ ]:
tad_1['y'].hist()

In [ ]:
tad_1.loc[ tad_1['y'] < 600000 ]['y'].hist()

In [ ]:
(tad_1.loc[ tad_1['y'] < 600000 ].shape[0] / tad_1.shape[0] ) * 100

In [ ]:
tad_1 = tad_1.loc[ tad_1['y'] < 600000 ].reset_index(drop=True)

In [ ]:
tad_1

In [ ]:
tad_1.shape

## Partición de datos

In [ ]:
Xt , Xv , yt , yv = train_test_split( tad_1[varc] , tad_1['y'] , train_size=0.7 )

In [ ]:
Xt.shape, yt.shape[0] , Xv.shape, yv.shape[0] 

## Modelo / Entrenamiento - Regresión Lineal

In [ ]:
modelo_1 = LinearRegression(n_jobs=-1)

In [ ]:
modelo_1.fit( Xt , yt )

## Parámetros del modelo

In [ ]:
modelo_1.intercept_ , modelo_1.coef_

## Evaluación del modelo

In [ ]:
mean_absolute_error( y_pred = modelo_1.predict(Xt) , y_true = yt )

In [ ]:
mean_absolute_error( y_pred = modelo_1.predict(Xv) , y_true = yv )

## Visualización de predicciones

In [ ]:
yv.hist()
plt.hist( modelo_1.predict(Xv))

In [ ]:
Xv['y'] = yv
Xv['y^'] = modelo_1.predict(Xv[varc])

In [ ]:
Xv

In [ ]:
sns.displot( yv ,  kde_kws = {'cumulative':True} )
sns.displot( Xv['y^'] ,  kde_kws = {'cumulative':True} ) # error en hist

In [ ]:
Xv['error'] = Xv['y^'] - Xv['y']

In [ ]:
Xv['error'].hist()

In [ ]:
Xv.head()

# Modelo (2) : Clasificación

In [ ]:
tad_2.head()

In [ ]:
tad_2.y2.value_counts()

In [ ]:
tad_2.y2.value_counts(True)*100

In [ ]:
X = tad_2[varc]
y = tad_2['y2']

## Partición de datos

In [ ]:
Xt , Xv , yt , yv = train_test_split( tad_2[varc] , tad_2['y2'] , train_size=0.7 )

In [ ]:
Xt.shape, yt.shape[0] , Xv.shape, yv.shape[0] 

## Modelo / Entrenamiento - Regresión Logística

In [ ]:
modelo_2 = LogisticRegression()

In [ ]:
modelo_2.fit( Xt , yt )

In [ ]:
modelo_2.coef_ , modelo_2.intercept_

## Evaluación de modelo

In [ ]:
pd.DataFrame(modelo_2.predict_proba( Xv ))

In [ ]:
pd.DataFrame(modelo_2.predict( Xv )).value_counts()

In [ ]:
plot_roc_curve( y_true=yt , y_probas= modelo_2.predict_proba(Xt) , curves='macro' )
plot_roc_curve( y_true=yv , y_probas= modelo_2.predict_proba(Xv) , curves='macro' )

In [ ]:
metricas( modelo_2 , Xv, yv )